In [1]:
import json
import spacy
from spacy import displacy
from spacy.training import Example
from spacy.util import minibatch, compounding
import random
import time
import pickle

In [2]:
# Load JSON data
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [4]:
file_1 = load_json("C:/Users/user/OneDrive/Desktop/json/file1.json")
file_2 = load_json("C:/Users/user/OneDrive/Desktop/json/file2.json")
file_3 = load_json("C:/Users/user/OneDrive/Desktop/json/file3.json")
file_4 = load_json("C:/Users/user/OneDrive/Desktop/json/file4.json")
file_5 = load_json("C:/Users/user/OneDrive/Desktop/json/file5.json")
file_6 = load_json("C:/Users/user/OneDrive/Desktop/json/file6.json")
file_7 = load_json("C:/Users/user/OneDrive/Desktop/json/file7.json")
file_8 = load_json("C:/Users/user/OneDrive/Desktop/json/file8.json")
file_9 = load_json("C:/Users/user/OneDrive/Desktop/json/file9.json")
file_10 = load_json("C:/Users/user/OneDrive/Desktop/json/file10.json")
file_11 = load_json("C:/Users/user/OneDrive/Desktop/json/file11.json")
file_12 = load_json("C:/Users/user/OneDrive/Desktop/json/file12.json")
file_13 = load_json("C:/Users/user/OneDrive/Desktop/json/file13.json")
file_14 = load_json("C:/Users/user/OneDrive/Desktop/json/file14.json")
file_15 = load_json("C:/Users/user/OneDrive/Desktop/json/file15.json")


In [5]:
# Convert JSON data to training format
def convert_data(data):
    training_data = []
    for item in data.get("annotations", []):
        if item is None:
            continue
        try:
            text, annotations = item
            if not text or not annotations or "entities" not in annotations:
                continue
            entities = [(start, end, label) for start, end, label in annotations["entities"]]
            training_data.append((text, {"entities": entities}))
        except ValueError:
            continue
    return training_data

In [6]:
# Combine training data from all files
training_data = []
files = [file_1, file_2, file_3, file_4, file_5, file_6, file_7, file_8, file_9, file_10, file_11, file_12, file_13, file_14, file_15]
for file in files:
    training_data.extend(convert_data(file))

In [7]:
# Split data into training and validation sets
random.shuffle(training_data)
split = int(len(training_data) * 0.8)
train_data = training_data[:split]
val_data = training_data[split:]

In [8]:
# Create a blank spaCy model
nlp = spacy.blank("en")

# Create the NER component and add it to the pipeline
ner = nlp.add_pipe("ner")

In [9]:
# Add new labels to the NER component
for _, annotations in training_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# Disable other pipeline components (if any)
pipe_exceptions = ["ner"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [10]:
# Training the NER model
epochs = 10
batch_size = 32

with nlp.disable_pipes(*unaffected_pipes):
    optimizer = nlp.begin_training()
    for epoch in range(epochs):
        start_time = time.time()
        random.shuffle(train_data)
        losses = {}
        batches = minibatch(train_data, size=batch_size)
        for batch in batches:
            texts, annotations = zip(*batch)
            examples = [Example.from_dict(nlp.make_doc(text), ann) for text, ann in zip(texts, annotations)]
            nlp.update(examples, drop=0.5, losses=losses)

        # Calculate validation loss
        val_loss = 0.0
        for text, ann in val_data:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, ann)
            updates = ner.update([example], sgd=optimizer, drop=0.0)
            val_loss += updates["ner"]

        end_time = time.time()
        print(f"Epoch {epoch+1}/{epochs}")
        print(f" - {len(train_data)}/{len(train_data)} - {round(end_time - start_time, 2)}s - loss: {losses['ner']} - val_loss: {val_loss}")


c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Police: Man killed ex grandmother after son wasn&a..." with entities "[(54, 58, 'NUMBER')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "ÛÏ@BBCEngland: A burst water main causes major fl..." with entities "[(4, 14, 'LOCATION'), (48, 56, 'DISASTER'), (62, 7...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\training\io

Epoch 1/10
 - 1884/1884 - 51.97s - loss: 9628.910990601822 - val_loss: 1075.8171350032183
Epoch 2/10
 - 1884/1884 - 47.76s - loss: 4095.7652194155335 - val_loss: 665.297682274841
Epoch 3/10
 - 1884/1884 - 32.78s - loss: 3995.375361298474 - val_loss: 475.8388749362053
Epoch 4/10
 - 1884/1884 - 32.95s - loss: 3800.0748378602516 - val_loss: 293.4221388732337
Epoch 5/10
 - 1884/1884 - 39.77s - loss: 3779.830586144206 - val_loss: 258.71109213840674
Epoch 6/10
 - 1884/1884 - 32.92s - loss: 3698.2269860252663 - val_loss: 239.49909336230488
Epoch 7/10
 - 1884/1884 - 41.72s - loss: 3484.9706731401384 - val_loss: 169.46625107187953
Epoch 8/10
 - 1884/1884 - 29.08s - loss: 3408.501753074768 - val_loss: 144.83177930335629
Epoch 9/10
 - 1884/1884 - 33.09s - loss: 3268.5806395705886 - val_loss: 124.58374495510343
Epoch 10/10
 - 1884/1884 - 28.56s - loss: 3238.5478811718262 - val_loss: 111.40453979887009


In [11]:
# Save the trained model using pickle
with open("model.pkl", "wb") as f:
    pickle.dump(nlp, f)


# Load the model using pickle
with open("model.pkl", "rb") as f:
    nlp = pickle.load(f)


In [14]:
# Input text
test_text = '''Just happened a terrible car crash
Heard about #earthquake is different cities, stay safe everyone.
there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all
Apocalypse lighting. #Spokane #wildfires
Typhoon Soudelor kills 28 in China and Taiwan
Arson suspect linked to 30 fires caught in Northern California http://t.co/HkFPyNb4PS
Horrible Accident Man Died In Wings of Airplane 
'''

# Process the text
doc = nlp(test_text)

In [15]:
# Define custom colors for entities
colors = {
    "DISASTER": "pink",
    "LOCATION": "#8da0cb",
    "DATE": "yellow",
    "NUMBER": "grey",
}

# Create options for displaCy with custom colors
options = {"colors": colors}

# Visualize the recognized entities with custom colors
displacy.render(doc, style="ent", jupyter=True, options=options)